## H-Bonding with Temperature

Since we're unsure whether the Cp spike we're observing at 225 K is due to the solvent or the molecule, we need to investigate the average hydrogen bonds formed in each replica simulation. This will confirm if there is a significant change in the number of hydrogen bonds at low temperatures, indicating that the terphenyl molecule may be giving rise to the large heat capacity spike.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h_bonds
import matplotlib.pyplot as plt
import numpy as np
import pymbar
import heat_capacity
from multiprocessing import Pool

plt.style.use('ggplot')

In [ ]:
remd_trajs = h_bonds.REMD_trajectories("/mnt/summit/simulations/octamer_Rchiral/RHH/remd_sim/200K_to_350K", "npt", "whole.xtc","sim", "/mnt/summit/simulations/octamer_Rchiral/RHH/remd_sim/200K_to_350K/sim0/berendsen.gro", np = 1)

Replica 0
npt.whole.xtc
npt.part0002.whole.xtc
npt.part0003.whole.xtc
npt.part0004.whole.xtc
Replica 1
npt.whole.xtc
npt.part0002.whole.xtc
npt.part0003.whole.xtc
npt.part0004.whole.xtc
Replica 2
npt.whole.xtc
npt.part0002.whole.xtc
npt.part0003.whole.xtc
npt.part0004.whole.xtc
Replica 3
npt.whole.xtc
npt.part0002.whole.xtc
npt.part0003.whole.xtc
npt.part0004.whole.xtc
Replica 4
npt.whole.xtc
npt.part0002.whole.xtc
npt.part0003.whole.xtc
npt.part0004.whole.xtc
Replica 5
npt.whole.xtc
npt.part0002.whole.xtc
npt.part0003.whole.xtc
npt.part0004.whole.xtc
Replica 6
npt.whole.xtc
npt.part0002.whole.xtc
npt.part0003.whole.xtc
npt.part0004.whole.xtc
Replica 7
npt.whole.xtc
npt.part0002.whole.xtc
npt.part0003.whole.xtc
npt.part0004.whole.xtc
Replica 8
npt.whole.xtc
npt.part0002.whole.xtc
npt.part0003.whole.xtc
npt.part0004.whole.xtc
Replica 9
npt.whole.xtc
npt.part0002.whole.xtc
npt.part0003.whole.xtc
npt.part0004.whole.xtc
Replica 10
npt.whole.xtc
npt.part0002.whole.xtc
npt.part0003.whole.xtc

In [ ]:
h_bond_finder = h_bonds.HydrogenBondFinder(remd_trajs.trajs[0][0], remd_trajs.trajs[0][0].top)
h_bond_finder.get_donors()
h_bond_finder.get_acceptors()
n_h_bonds, h_bond_ids = h_bond_finder.get_hydrogen_bonds(remd_trajs.trajs[0])

In [ ]:
pool = Pool(4)

In [ ]:
n_h_bonds_remd, h_bonds_remd = zip(*pool.map(h_bond_finder.get_hydrogen_bonds, remd_trajs.trajs))

In [ ]:
n_h_bonds_remd = np.array(n_h_bonds_remd)
h_bonds_remd = h_bonds_remd

In [ ]:
temps = np.array(remd_trajs.temps)

In [ ]:
plt.scatter(temps, np.mean(n_h_bonds_remd, axis = 1))
plt.ylabel("# Hydrogen Bonds")
plt.xlabel("Replica Temperature")

## H-bond analysis using ensemble averaging

Now that we've extracted the number of H-bonds per frame at each temperature we can use ensemble reweighting to get a continuous function of the number of hydrogen bonds over the range of replica temperatures.

In [ ]:
# Extract Potential Energies from each simulation
sim_dir_name = "sim"
path = "/mnt/summit/simulations/octamer_Rchiral/RHH/remd_sim/200K_to_350K/"
n_replicas = 40

energies, temps = heat_capacity.get_energies(sim_dir_name, path, n_replicas)

In [ ]:
u_kln, n_samples, t_list, betas = heat_capacity.construct_u_kln_matrix(temps, energies, add_temps = np.linspace(200, 350, 200))

In [ ]:
h_bonds_kln = np.zeros([240, 240, 3608])
for k in range(n_h_bonds_remd.shape[0]):
    for l in range(h_bonds_kln.shape[0]):
        h_bonds_kln[k,l,:] = n_h_bonds_remd[k]

In [ ]:
for i in range(h_bonds_kln.shape[0]):
    print(h_bonds_kln)

In [ ]:
n_samples[:40] = h_bonds_kln.shape[-1]

In [ ]:
mbar_h_bonds = pymbar.MBAR(u_kln[:,:,:3608], n_samples, verbose = True, relative_tolerance = 1e-10, initial_f_k= None, maximum_iterations=1000)

In [ ]:
results = mbar_h_bonds.computeExpectations(h_bonds_kln, state_dependent=True, return_dict=True)

h_bonds_mu = results["mu"]
h_bonds_sigma = results["sigma"]

In [ ]:
n_replicas = 40
plt.figure(figsize=[8,4], dpi=600)
# plt.scatter(t_list[:n_replicas], h_bonds_mu[:n_replicas])
plt.plot(t_list[n_replicas:], h_bonds_mu[n_replicas:], c="gray")
plt.errorbar(t_list[:n_replicas], h_bonds_mu[:n_replicas], yerr=2*h_bonds_sigma[:n_replicas], fmt=".")
plt.fill_between(t_list[n_replicas:], h_bonds_mu[n_replicas:]-2*h_bonds_sigma[n_replicas:],  h_bonds_mu[n_replicas:]+2*h_bonds_sigma[n_replicas:], color="gray", alpha = 0.3)


plt.ylabel("# Hydrogen Bonds")
plt.xlabel("Temperature (K)")